# Data exploration of tapis v2 

# Modified DFs
These dataframes (unless specified) are created below

- `valid_emails` : df of JsonArray containiung value in `value.email`

- `valid_email_list` : list of valid emails

- `non_read_users` : metadataperms, but only READ/WRITE or ALL

- `job_event_perms` : job events and job permissions merge on job_id

- `email_username_combo` : combination of `value.username` and `value.email`

- `num_jobs` : number of jobs for each `parameter.Creator` with username and email

- `projJob_projFile_jobAll_emailUser` : 
    - merging of `jsonArray_df` where `name=projectJob` and `jsonArray_df` where `name=projectFile` (`on=`value.projectUuid`)
    - then merge with `jobs_all_df` (`left_on=jobUuid` and `right_on=uuid`) 
    - then merged with `email_username_combo` (`left_on=parameters.Creator` and `right_on=value.username`)

- `projJob_projFile_jobAll_emailUser_groupby` : Prior df, but grouped by `parameters.Creators` and sorted by `value.projectUuid`

# Imports

In [1]:
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
import json

# Read files

In [2]:
job_events_df = pd.read_json('/mnt/data2/Projects/vdjserver/vdjserverJobEvents.json')
job_permissions_df = pd.read_json('/mnt/data2/Projects/vdjserver/vdjserverJobPermissions.json')
jobs_all_df = pd.read_json('/mnt/data2/Projects/vdjserver/vdjserverJobs_all.json')
jobs_all_df['parameters.Creator'] = jobs_all_df['parameters'].apply(lambda x: json.loads(x).get('Creator', np.nan) if x else np.nan)
metadata_perms_df = pd.read_json('/mnt/data2/Projects/vdjserver/vdjserverMetadataPermissions.json')

with open('/mnt/data2/Projects/vdjserver/vdjserverJsonArrayFeb042025.json', 'r') as f:
    jsonarray = json.load(f)

In [3]:
# probably not the best way of doing this
# max_level must be 1, otherwise df is too large for system
# I end up using pd.read_json later on, but the creation of valid email list uses this.
jsonarray_df = pd.json_normalize(jsonarray, max_level=1)


# View dataframes

In [4]:
job_events_df.head(2)

,id,created,created_by,description,ip_address,status,tenant_id,job_id,transfertask,uuid
0,5476,2014-03-31 14:48:10,jfonner,Job accepted and queued for submission.,129.114.60.167,PENDING,vdjserver.org,142,NaN,None
1,5477,2014-03-31 14:48:20,jfonner,No inputs for the given job. Skipping staging,129.114.60.167,STAGED,vdjserver.org,142,NaN,None


In [5]:
job_permissions_df.head(2)

,id,job_id,last_updated,permission,tenant_id,username
0,3,783,2014-07-11 09:11:00,READ_EXECUTE,vdjserver.org,mlevin
1,4,783,2014-07-11 09:33:02,ALL,vdjserver.org,wscarbor2


In [6]:
jobs_all_df.head(2)

,id,name,tenant_id,tenant_queue,owner,roles,system_id,app_id,app_uuid,status,...,remote_ended,remote_outcome,remote_submit_retries,remote_status_checks,failed_status_checks,last_status_check,blocked_count,visible,update_token,parameters.Creator
0,503865,My Job 24-Jan-2025 8:57:09 pm,vdjserver.org,aloe.jobq.vdjserver.org.submit.DefaultQueue,vdj,"Internal/VDJ_vdj_keycloak_PRODUCTION,Internal/...",ls6.tacc.utexas.edu,repcalc-ls6-2.0u8,6306626279335587345-242ac119-0001-005,FINISHED,...,2025-01-25 15:43:33.007,FINISHED,0,192,0,2025-01-25 15:43:33.001,0,1,eb27e311-4a37-4aeb-b649-056704dd2711,schristley
1,503859,My Job 23-Jan-2025 3:05:30 pm,vdjserver.org,aloe.jobq.vdjserver.org.submit.DefaultQueue,vdj,"Internal/VDJ_vdj_keycloak_PRODUCTION,Internal/...",ls6.tacc.utexas.edu,igblast-ls6-1.20u6,1936847182374244846-242ac119-0001-005,FINISHED,...,2025-01-24 04:18:20.211,FINISHED,0,112,0,2025-01-24 04:18:20.201,9,1,5e2528fd-25d6-4473-9287-6a67a8de8391,schristley


In [7]:
metadata_perms_df.head(2)

,id,last_updated,permission,username,uuid,tenant_id
0,5,2014-01-21 17:00:11,READ_WRITE,test3,0001389977207738-5056a550b8-0001-012,vdjserver.org
1,6,2014-01-21 17:00:41,READ_WRITE,test4,0001389977207738-5056a550b8-0001-012,vdjserver.org


# Explore

## find valid emails

In [8]:
# Extract list of valid emails
domains = ['test', 'email']
tlds = ['.edu', '.com', '.org']
special = ['#$^&&*)(_+{{7*7}}||\\[:&lt;/&gt;,.?']
bogus_domains = [d+t for d in domains for t in tlds]
bogus_domains = bogus_domains + special

valid_emails = jsonarray_df[jsonarray_df['value.email'].notna()]
valid_emails = valid_emails[valid_emails['value.email'].str.split('@').apply(len)==2]
valid_emails = valid_emails[valid_emails['value.email'].apply(lambda x: x.split('@')[1] not in bogus_domains)]

valid_email_list = [e for e in valid_emails['value.email'].unique() if e.split('@')[1].lower() not in bogus_domains]

# Construct dataframe of email-username combinations
email_username_combo = valid_emails.loc[:,['value.username', 'value.email']]
email_username_combo = email_username_combo[email_username_combo['value.username'].notna()].drop_duplicates()
email_username_combo['value.username'].value_counts()

# valid_emails.loc[[478159,478163,478174,568135,568139],['value.username', 'value.email', 'lastUpdated']]

value.username
att23562             3
diksha28             2
paulatataru          1
frankabuytenhuijs    1
mbartl               1
                    ..
pejvak               1
jinhyun              1
el-hadi              1
saleach              1
cdallett             1
Name: count, Length: 1670, dtype: int64

## vdjserverJsonArrayFeb042025.json

`name` contains `profile`, `feedback` when restricted on `value.email` as not `NaN`

In [10]:
# jsonarray_df.groupby(by=['owner']).count().sort_values(by=['created'], ascending=False).to_csv('./csv/owner_created_count_sorted.csv')

### json projectFile

In [11]:
jsonarray_projectFile = pd.read_json('/mnt/data2/Projects/vdjserver/vdjserverJsonArrayFeb042025.json')
jsonarray_projectFile = jsonarray_projectFile[jsonarray_projectFile['name'] == 'projectFile']

jsonarray_projectFile['_id.$oid'] = jsonarray_projectFile['value'].apply(lambda x: x.get('$oid', None))
jsonarray_projectFile['associationIds.0'] = jsonarray_projectFile['value'].apply(lambda x: x[0] if isinstance(x, list) and len(x)>0 else None)
jsonarray_projectFile['associationIds.1'] = jsonarray_projectFile['value'].apply(lambda x: x[1] if isinstance(x, list) and len(x)>1 else None)
jsonarray_projectFile['value.projectUuid'] = jsonarray_projectFile['value'].apply(lambda x: x.get('projectUuid', None))
jsonarray_projectFile['value.type'] = jsonarray_projectFile['value'].apply(lambda x: x.get('type', None))
jsonarray_projectFile.rename(columns={'uuid':'fileUuid'}, inplace=True)
jsonarray_projectFile.drop(columns=['_id', 'associationIds', 'value'], inplace=True)

jsonarray_projectFile

,fileUuid,owner,tenantId,schemaId,internalUsername,lastUpdated,name,created,_id.$oid,associationIds.0,associationIds.1,value.projectUuid,value.type
53,0001395955349445-5056a550b8-0001-012,vdjauth,vdjserver.org,NaN,NaN,2014-03-27T16:22:29.444-05:00,projectFile,2014-03-27T16:22:29.444-05:00,None,None,None,,
54,0001396029083309-5056a550b8-0001-012,vdjauth,vdjserver.org,NaN,NaN,2014-03-28T12:51:23.309-05:00,projectFile,2014-03-28T12:51:23.309-05:00,None,None,None,0001395346788177-5056a550b8-0001-012,uploaded
55,0001396029805022-5056a550b8-0001-012,vdjauth,vdjserver.org,NaN,NaN,2014-03-28T13:03:25.022-05:00,projectFile,2014-03-28T13:03:25.022-05:00,None,None,None,0001395346788177-5056a550b8-0001-012,uploaded
56,0001396030144907-5056a550b8-0001-012,vdjauth,vdjserver.org,NaN,NaN,2014-03-28T13:09:04.907-05:00,projectFile,2014-03-28T13:09:04.907-05:00,None,None,None,0001395346788177-5056a550b8-0001-012,uploaded
57,0001396039988083-5056a550b8-0001-012,vdjauth,vdjserver.org,NaN,NaN,2014-03-28T15:53:08.083-05:00,projectFile,2014-03-28T15:53:08.083-05:00,None,None,None,0001395346788177-5056a550b8-0001-012,uploaded
...,...,...,...,...,...,...,...,...,...,...,...,...,...
607085,5338423137409494545-242ac118-0001-012,vdj,vdjserver.org,NaN,NaN,2025-01-13T16:40:40.230-06:00,projectFile,2025-01-13T16:14:09.079-06:00,None,None,None,5456400192359305711-242ac118-0001-012,None
607086,1335427718191574545-242ac118-0001-012,vdj,vdjserver.org,NaN,NaN,2025-01-13T16:40:40.230-06:00,projectFile,2025-01-13T16:15:42.281-06:00,None,None,None,5456400192359305711-242ac118-0001-012,None
607087,1840700597200490991-242ac118-0001-012,vdj,vdjserver.org,NaN,NaN,2025-01-13T16:40:43.277-06:00,projectFile,2025-01-13T16:16:56.231-06:00,None,None,None,5456400192359305711-242ac118-0001-012,None
607088,5023614960920170991-242ac118-0001-012,vdj,vdjserver.org,NaN,NaN,2025-01-13T16:40:43.281-06:00,projectFile,2025-01-13T16:18:10.339-06:00,None,None,None,5456400192359305711-242ac118-0001-012,None


In [12]:
jsonarray_projectFile.groupby('value.projectUuid').nunique()

,fileUuid,owner,tenantId,schemaId,internalUsername,lastUpdated,name,created,_id.$oid,associationIds.0,associationIds.1,value.type
value.projectUuid,,,,,,,,,,,,
,44,3,1,0,0,44,1,44,0,0,0,1
0001395346788177-5056a550b8-0001-012,16,1,1,0,0,16,1,16,0,0,0,1
0001396538655269-5056a550b8-0001-012,1,1,1,0,0,1,1,1,0,0,0,1
0001396562317882-5056a550b8-0001-012,2,1,1,0,0,2,1,2,0,0,0,1
0001396564389482-5056a550b8-0001-012,3,1,1,0,0,3,1,3,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
988793832798425581-242ac118-0001-012,24,1,1,0,0,24,1,24,0,0,0,0
991355144541573606-242ac114-0001-012,1,1,1,0,0,1,1,1,0,0,0,0
991430390416535060-242ac117-0001-012,1,1,1,0,0,1,1,1,0,0,0,0


### json projectJob

In [13]:
jsonarray_projectJob = pd.read_json('/mnt/data2/Projects/vdjserver/vdjserverJsonArrayFeb042025.json')
jsonarray_projectJob = jsonarray_projectJob[jsonarray_projectJob['name'] == 'projectJob']

jsonarray_projectJob['_id.$oid'] = jsonarray_projectJob['value'].apply(lambda x: x.get('$oid', None))
jsonarray_projectJob['associationIds.0'] = jsonarray_projectJob['value'].apply(lambda x: x[0] if isinstance(x, list) and len(x)>0 else None)
jsonarray_projectJob['associationIds.1'] = jsonarray_projectJob['value'].apply(lambda x: x[1] if isinstance(x, list) and len(x)>1 else None)
jsonarray_projectJob['value.projectUuid'] = jsonarray_projectJob['value'].apply(lambda x: x.get('projectUuid', None))
jsonarray_projectJob['value.jobUuid'] = jsonarray_projectJob['value'].apply(lambda x: x.get('jobUuid', None))
jsonarray_projectJob.drop(columns=['_id', 'associationIds', 'value'], inplace=True)

jsonarray_projectJob

,uuid,owner,tenantId,schemaId,internalUsername,lastUpdated,name,created,_id.$oid,associationIds.0,associationIds.1,value.projectUuid,value.jobUuid
250,0001400192074855-5056a550b8-0001-012,vdj,vdjserver.org,NaN,NaN,2014-05-15T17:14:34.855-05:00,projectJob,2014-05-15T17:14:34.855-05:00,None,None,None,0001399309581559-5056a550b8-0001-012,0001399315558601-5056a550b8-0001-007
252,0001400254373114-5056a550b8-0001-012,vdj,vdjserver.org,NaN,NaN,2014-05-16T10:32:53.114-05:00,projectJob,2014-05-16T10:32:53.114-05:00,None,None,None,0001400250478554-5056a550b8-0001-012,0001400254372814-5056a550b8-0001-007
253,0001400273862423-5056a550b8-0001-012,vdj,vdjserver.org,NaN,NaN,2014-05-16T15:57:42.423-05:00,projectJob,2014-05-16T15:57:42.423-05:00,None,None,None,0001400250478554-5056a550b8-0001-012,0001400273862119-5056a550b8-0001-007
254,0001400274448495-5056a550b8-0001-012,vdj,vdjserver.org,NaN,NaN,2014-05-16T16:07:28.494-05:00,projectJob,2014-05-16T16:07:28.494-05:00,None,None,None,0001400250478554-5056a550b8-0001-012,0001400274448320-5056a550b8-0001-007
256,0001400274714655-5056a550b8-0001-012,vdj,vdjserver.org,NaN,NaN,2014-05-16T16:11:54.655-05:00,projectJob,2014-05-16T16:11:54.655-05:00,None,None,None,0001400250478554-5056a550b8-0001-012,0001400274714490-5056a550b8-0001-007
...,...,...,...,...,...,...,...,...,...,...,...,...,...
606711,5097479121213854191-242ac118-0001-012,vdj,vdjserver.org,NaN,NaN,2025-01-08T12:13:35.460-06:00,projectJob,2025-01-08T12:13:35.460-06:00,None,None,None,6589143665654501871-242ac118-0001-012,ad02cb34-250e-48cb-a06e-973e431b62ee-007
607090,1948444895656078865-242ac118-0001-012,vdj,vdjserver.org,NaN,NaN,2025-01-13T16:44:05.995-06:00,projectJob,2025-01-13T16:44:05.995-06:00,None,None,None,5456400192359305711-242ac118-0001-012,c0ab5f4a-97b0-4dc3-93e8-0908c95cb3a4-007
607329,1819643224410746385-242ac118-0001-012,vdj,vdjserver.org,NaN,NaN,2025-01-20T03:06:57.762-06:00,projectJob,2025-01-20T03:06:57.762-06:00,None,None,None,5199144433477554666-242ac116-0001-012,773a5cb7-b369-4517-a221-83d57e3899e5-007
607330,2845695380777266705-242ac118-0001-012,vdj,vdjserver.org,NaN,NaN,2025-01-23T15:05:59.570-06:00,projectJob,2025-01-23T15:05:59.570-06:00,None,None,None,5456400192359305711-242ac118-0001-012,9188bf80-e868-4e05-a6b4-308c044108d7-007


In [14]:
jsonarray_projectJob.groupby('value.projectUuid').nunique()

,uuid,owner,tenantId,schemaId,internalUsername,lastUpdated,name,created,_id.$oid,associationIds.0,associationIds.1,value.jobUuid
value.projectUuid,,,,,,,,,,,,
0001399309581559-5056a550b8-0001-012,1,1,1,0,0,1,1,1,0,0,0,1
0001400250478554-5056a550b8-0001-012,25,1,1,0,0,25,1,25,0,0,0,25
0001401392421049-5056a550b8-0001-012,9,1,1,0,0,9,1,9,0,0,0,9
0001401393981043-5056a550b8-0001-012,1,1,1,0,0,1,1,1,0,0,0,1
0001401395232432-5056a550b8-0001-012,3,1,1,0,0,3,1,3,0,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...
976788537735179795-242ac118-0001-012,4,1,1,0,0,4,1,4,0,0,0,4
988793832798425581-242ac118-0001-012,4,1,1,0,0,4,1,4,0,0,0,4
991355144541573606-242ac114-0001-012,1,1,1,0,0,1,1,1,0,0,0,1


## combine projectJob jobAll and emailUsernameCombo

In [15]:
# from jsonArray name=projectJob combined with jobs_all json file 
projJob_projFile_jobAll_emailUser = jsonarray_projectJob.merge(jsonarray_projectFile, on='value.projectUuid')\
                                                        .merge(jobs_all_df, left_on='value.jobUuid', right_on='uuid')\
                                                        .merge(email_username_combo, left_on='parameters.Creator', right_on='value.username')

                                                        


# extract all columns
cols = projJob_projFile_jobAll_emailUser.columns.to_list()

# define custom agg function for email and username when only 1 present, print out name, otherwise unique count
def custom_agg(series):
    unique_values = series.unique()
    if len(unique_values) == 1:
        return unique_values[0]
    else:
        return len(unique_values)

# define columns to use this function 
user_cols = {'value.username': custom_agg,
             'value.email': custom_agg}

# create agg_dict
agg_dict = {col: 'nunique' for col in cols if col not in user_cols}
agg_dict.update(user_cols)

# groupby
projJob_projFile_jobAll_emailUser_groupby = projJob_projFile_jobAll_emailUser.groupby('parameters.Creator').agg(agg_dict).sort_values('value.projectUuid', ascending=False)

# save
# projJob_projFile_jobAll_emailUser_groupby.to_csv('./csv/jsonArrayProjectJob.merge.jsonArrayProjectFile_on=projectUuid'+
#                                                  '.merge.jobsAll_on=jobUuid&uuid'+
#                                                  '.merge.emailUserCombo_on=Creator&username'+
#                                                  '.groupby=Creators'+
#                                                  '.sorted=projectUuid.csv')
projJob_projFile_jobAll_emailUser_groupby

,uuid_x,owner_x,tenantId_x,schemaId_x,internalUsername_x,lastUpdated_x,name_x,created_x,_id.$oid_x,associationIds.0_x,...,remote_submit_retries,remote_status_checks,failed_status_checks,last_status_check,blocked_count,visible,update_token,parameters.Creator,value.username,value.email
parameters.Creator,,,,,,,,,,,,,,,,,,,,,
vdj-test1,84,2,1,0,0,84,1,84,0,0,...,2,3,1,2,2,1,84,1,vdj-test1,vdjserver@utsouthwestern.edu
schristley,456,1,1,0,0,456,1,456,0,0,...,1,184,2,338,13,1,456,1,schristley,scott.christley@utsouthwestern.edu
scott_public,305,1,1,0,0,305,1,305,0,0,...,1,236,2,222,12,1,305,1,scott_public,scott.christley@utsouthwestern.edu
victorialopez,120,1,1,0,0,120,1,120,0,0,...,1,49,1,84,3,1,120,1,victorialopez,victorialopez@isciii.es
scott_ab,51,1,1,0,0,51,1,51,0,0,...,1,50,1,18,7,1,51,1,scott_ab,scott.christley@utsouthwestern.edu
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
baoduong,1,1,1,0,0,1,1,1,0,0,...,1,1,1,1,1,1,1,1,baoduong,baoduong@uab.edu
zhangcy,18,1,1,0,0,18,1,18,0,0,...,1,6,1,14,13,1,18,1,zhangcy,caiyan505@126.com
zhe.sang,1,1,1,0,0,1,1,1,0,0,...,1,1,1,1,1,1,1,1,zhe.sang,zhe.sang@gmail.com


In [16]:
# Pull out stats required
stats_cols = ['value.email', 'value.projectUuid', 'fileUuid', 'value.jobUuid', ]
# projJob_projFile_jobAll_emailUser_groupby[stats_cols].to_csv('./csv/reduced_stats.csv')
projJob_projFile_jobAll_emailUser_groupby[stats_cols].head()

,value.email,value.projectUuid,fileUuid,value.jobUuid
parameters.Creator,,,,
vdj-test1,vdjserver@utsouthwestern.edu,84,84,84
schristley,scott.christley@utsouthwestern.edu,64,7097,456
scott_public,scott.christley@utsouthwestern.edu,32,3242,305
victorialopez,victorialopez@isciii.es,27,218,120
scott_ab,scott.christley@utsouthwestern.edu,23,4175,51


In [17]:
# tests used when pulling '_id's, 'associationIds's, and 'value's

# print(set([str(e.keys()) for e in jsonarray_projectJob['_id']]))
# print(set(len(e) for e in jsonarray_projectJob['associationIds']))
# print(set([str(e.keys()) for e in jsonarray_projectJob['value']]))

## vdjserverMetadataPermissions.json

In [18]:
metadata_perms_df.groupby(['permission']).count()

,id,last_updated,username,uuid,tenant_id
permission,,,,,
ALL,6364,6364,6364,6364,6364
READ,46912,46912,46912,46912,46912
READ_WRITE,707319,707319,707319,707319,707319


In [19]:
non_read_users = metadata_perms_df[(metadata_perms_df['permission'] == 'ALL') | (metadata_perms_df['permission'] == 'READ_WRITE')]
# non_read_users.groupby(by=['username']).count().sort_values(by=['uuid'],ascending=False)
non_read_users.value_counts('username')

username
schristley      242530
scott_test1      62528
itoby            54506
lgcowell         51657
scott_public     43133
                 ...  
zktuong              1
zhonghuang           1
zhanxw               1
zerufael             1
a.sooda              1
Name: count, Length: 1058, dtype: int64

## counting the number of jobs
- From vdjserverJobs_all.json the parameters.Creator var was counted, 
- This count left joined with a dataframe containing email and username combinations. 
    - These combos were made from vdjserverJsonArrayFeb042025.json by finding valid emails by filtering out any @test or @email with varying top level domains, and checking if the address can be split on @ with a len of 2. The injection bit was also removed

In [20]:
# cant do this because owner is usually vdj since tacc account needed. 
# we will replace owner with parameters.creator
num_jobs = jobs_all_df['parameters.Creator'].value_counts().to_frame().reset_index().rename(columns={'count':'num_jobs'})
num_jobs = num_jobs.merge(email_username_combo, how='left', left_on='parameters.Creator', right_on='value.username')
# num_jobs.to_csv('./csv/num_jobs_by_parameters.Creator_w_username_equal_to_parameters.Creator.csv')
num_jobs


,parameters.Creator,num_jobs,value.username,value.email
0,vdj,2552,NaN,NaN
1,scott_test1,643,scott_test1,scott.christley@utsouthwestern.edu
2,schristley,592,schristley,scott.christley@utsouthwestern.edu
3,scott_public,478,scott_public,scott.christley@utsouthwestern.edu
4,victorialopez,175,victorialopez,victorialopez@isciii.es
...,...,...,...,...
303,jivdj17,1,jivdj17,xuhuai.ji@stanford.edu
304,pingpingzheng,1,pingpingzheng,zhengpp@stanford.edu
305,minici.claudia,1,minici.claudia,minici.claudia@hsr.it
306,nianbinli,1,nianbinli,linianbin97@tmu.edu.cn


# jobsAll & jobsEvents

In [21]:
jobs_all_and_events_df = jobs_all_df.merge(job_events_df, right_on='job_id', left_on='id')
jobs_all_and_events_df.head()

,id_x,name,tenant_id_x,tenant_queue,owner,roles,system_id,app_id,app_uuid,status_x,...,id_y,created_y,created_by,description,ip_address,status_y,tenant_id_y,job_id,transfertask,uuid_y
0,503865,My Job 24-Jan-2025 8:57:09 pm,vdjserver.org,aloe.jobq.vdjserver.org.submit.DefaultQueue,vdj,"Internal/VDJ_vdj_keycloak_PRODUCTION,Internal/...",ls6.tacc.utexas.edu,repcalc-ls6-2.0u8,6306626279335587345-242ac119-0001-005,FINISHED,...,23585634,2025-01-25 02:57:54,vdj,Job processing beginning,172.17.0.5,PENDING,vdjserver.org,503865,0.0,2ad89d35-7a9b-49f8-92fc-6ac58a616bc0-028
1,503865,My Job 24-Jan-2025 8:57:09 pm,vdjserver.org,aloe.jobq.vdjserver.org.submit.DefaultQueue,vdj,"Internal/VDJ_vdj_keycloak_PRODUCTION,Internal/...",ls6.tacc.utexas.edu,repcalc-ls6-2.0u8,6306626279335587345-242ac119-0001-005,FINISHED,...,23585635,2025-01-25 02:57:54,vdj,Identifying input files for staging,172.17.0.5,PROCESSING_INPUTS,vdjserver.org,503865,0.0,a59279d9-4e37-456f-bc14-b44ed6a150fd-028
2,503865,My Job 24-Jan-2025 8:57:09 pm,vdjserver.org,aloe.jobq.vdjserver.org.submit.DefaultQueue,vdj,"Internal/VDJ_vdj_keycloak_PRODUCTION,Internal/...",ls6.tacc.utexas.edu,repcalc-ls6-2.0u8,6306626279335587345-242ac119-0001-005,FINISHED,...,23585636,2025-01-25 02:57:54,vdj,ALL permission granted to schristley,172.17.0.3,PERMISSION_GRANT,vdjserver.org,503865,0.0,808c6bb4-8397-4462-bf5a-81c9f96b0b08-028
3,503865,My Job 24-Jan-2025 8:57:09 pm,vdjserver.org,aloe.jobq.vdjserver.org.submit.DefaultQueue,vdj,"Internal/VDJ_vdj_keycloak_PRODUCTION,Internal/...",ls6.tacc.utexas.edu,repcalc-ls6-2.0u8,6306626279335587345-242ac119-0001-005,FINISHED,...,23585637,2025-01-25 02:57:56,vdj,Transferring job input data to execution system,172.17.0.5,STAGING_INPUTS,vdjserver.org,503865,0.0,d230d601-4e32-45d1-a4f0-3c131b3572d6-028
4,503865,My Job 24-Jan-2025 8:57:09 pm,vdjserver.org,aloe.jobq.vdjserver.org.submit.DefaultQueue,vdj,"Internal/VDJ_vdj_keycloak_PRODUCTION,Internal/...",ls6.tacc.utexas.edu,repcalc-ls6-2.0u8,6306626279335587345-242ac119-0001-005,FINISHED,...,23585638,2025-01-25 02:57:58,vdj,Job input copy in progress: agave://data.vdjse...,172.17.0.5,STAGING_INPUTS,vdjserver.org,503865,560378492.0,9ee603a8-8777-4677-bbd3-0f55b2863676-028


In [22]:
job_event_perms = job_events_df.merge(job_permissions_df, on='job_id')
job_event_perms.merge(jobs_all_df, left_on='job_id', right_on='id')['parameters'][170000]

'{"UMIMinFrequency":"0.6","PreFilterStatisticsFlag":true,"MinimumQuality":20,"Workflow":"paired","Creator":"lcc294","ReversePrimer":"none","FindUniqueFlag":true,"SequenceFileTypes":"454","UMIMaxGap":"0.5","FilterFlag":true,"PostFilterStatisticsFlag":true,"SequenceReversePairedFilesMetadata":"1628993133380103705-242ac11c-0001-012","FindUniqueMaxNucleotides":20,"ForwardPrimer":"none","FindUniqueExclude":true,"Barcode":false,"MinimumLength":250,"SequenceForwardPairedFilesMetadata":"2433979131179691545-242ac11c-0001-012","UMIConsensus":false,"UMIMaxError":"0.1"}'